# Players dataframes with t-1 variables

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y pitchers. Debido a que es totalmente análogo al script para variables del mismo periodo $t$, se omitirán solo las explicaciones del código reutilizado.

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Creación de indicador de si el jugador es agente libre.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from matplotlib.colors import ListedColormap
from termcolor import colored
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')

In [3]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/Visualizations/Descriptive_statistics/Panel


In [4]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [5]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [6]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [7]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [8]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [9]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [10]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [11]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Per_10_Games/Hitting/hitting_'
pitching = 'Data/Statistics/Per_10_Games/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Transversal/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [12]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [13]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [14]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [15]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [16]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [17]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'TVS'],
      dtype='object')

In [18]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [19]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [20]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [21]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


Agreguemos los agentes libres en todas las temporadas que su contrato está vigente en lugar de solo tener observaciones en el año que firmaron. Para obervar cuántos datos se añadirán, veamos el tamaño inicial de las bases de datos.

In [22]:
period_t = period - 1
start_year = 2011
end_year = 2022
df_contracts = [None]*(period_t)

In [23]:
for year in range(1,period_t):
    
    max_year_contract = max(df_free_agents_copy[year]['Anios_de_contrato'])
    years = max_year_contract - 1
    df_contracts[year] = [None]*years
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = start_year + year + diff_t
        year_bound = end_year

        if real_year <= year_bound:
            df_contracts[year][incremento] = df_free_agents_copy[year][df_free_agents_copy[year]['Anios_de_contrato'] > diff_t]

In [24]:
for year in range(1,period_t):
    years = len(df_contracts[year])
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = start_year + year + diff_t
        year_bound = end_year

        if real_year <= year_bound:
            frames = [df_free_agents_copy[year + diff_t], df_contracts[year][incremento]]
            
            df_free_agents_copy[year + diff_t] = pd.concat(frames)
            
            df_free_agents_copy[year + diff_t].reset_index(drop = True, inplace = True)
            df_free_agents_copy[year + diff_t] = df_free_agents_copy[year + diff_t].sort_values(by = 'Jugador', ascending = True)

Veamos los resultados

In [25]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(243, 6)
(260, 6)
(293, 6)
(306, 6)
(182, 6)
(174, 6)
(189, 6)
(192, 6)
(203, 6)
(196, 6)


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [26]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [27]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [28]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [29]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [30]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [31]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [32]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-9
2
-7


In [33]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [34]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [35]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [36]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [37]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [38]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At-bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs-batted-in',
                            'HR':'Home-runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On-base-plus-slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on-base'})

In [39]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(515, 16)
(660, 16)
(728, 16)
(769, 16)
(801, 16)
(834, 16)
(807, 16)
(823, 16)
(834, 16)
(445, 16)
(752, 16)
(495, 16)


In [40]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    float64
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    float64
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At-bats                           834 non-null    float64
 6   Bateos                            834 non-null    float64
 7   Dobles                            834 non-null    float64
 8   Triples                           834 non-null    float64
 9   Home-runs                         834 non-null    float64
 10  Runs-batted-in                    834 non-null    float64
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [41]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'TVS'],
      dtype='object')

#### Pitchers

In [42]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike-outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [43]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(321, 16)
(424, 16)
(482, 16)
(536, 16)
(584, 16)
(586, 16)
(573, 16)
(641, 16)
(673, 16)
(537, 16)
(608, 16)
(595, 16)


In [44]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           586 non-null    object 
 1   Juegos            586 non-null    float64
 2   Juegos_iniciados  586 non-null    float64
 3   Inning_pitched    586 non-null    float64
 4   Bateos            586 non-null    float64
 5   Carreras          586 non-null    float64
 6   Carreras_ganadas  586 non-null    float64
 7   Walks             586 non-null    float64
 8   Strike-outs       586 non-null    float64
 9   Wins              586 non-null    float64
 10  Losses            586 non-null    float64
 11  Saves             586 non-null    float64
 12  WHIP              586 non-null    float64
 13  ERA               586 non-null    float64
 14  WAR               581 non-null    float64
 15  TVS               586 non-null    float64
dtypes: float64(15), object(1)
memory usage: 73.4

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(9*Inning \; Pitched)$
- CONTROL = $Walks/(9*Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [45]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike-outs']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike-outs']/df_pitching_copy[year]['Walks']

In [46]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(321, 19)
(424, 19)
(482, 19)
(536, 19)
(584, 19)
(586, 19)
(573, 19)
(641, 19)
(673, 19)
(537, 19)
(608, 19)
(595, 19)


In [47]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           482 non-null    object 
 1   Juegos            482 non-null    float64
 2   Juegos_iniciados  482 non-null    float64
 3   Inning_pitched    482 non-null    float64
 4   Bateos            482 non-null    float64
 5   Carreras          482 non-null    float64
 6   Carreras_ganadas  482 non-null    float64
 7   Walks             482 non-null    float64
 8   Strike-outs       482 non-null    float64
 9   Wins              482 non-null    float64
 10  Losses            482 non-null    float64
 11  Saves             482 non-null    float64
 12  WHIP              482 non-null    float64
 13  ERA               482 non-null    float64
 14  WAR               432 non-null    float64
 15  TVS               482 non-null    float64
 16  Dominio           481 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [48]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [49]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [50]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1013 non-null   object 
 1   Anio                     1013 non-null   object 
 2   Posicion                 1013 non-null   object 
 3   Acronimo                 1013 non-null   object 
 4   Sueldo_base              1013 non-null   int64  
 5   Bono_por_firma           1013 non-null   int64  
 6   Sueldo_regular           1013 non-null   int64  
 7   Sueldo_ajustado          1013 non-null   int64  
 8   Sueldo_porcentual        1013 non-null   float64
 9   Pago_efectivo            1013 non-null   int64  
 10  Valor_contrato_promedio  1013 non-null   int64  
 11  Anios_de_contrato        1013 non-null   int64  
 12  Valor_del_contrato       1013 non-null   int64  
 13  Ganancias                1013 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [51]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
     # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)

In [52]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike-outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike-outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [53]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike-outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike-outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [54]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [55]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [56]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [57]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
55  -inf
166  -inf
274  -inf

2022
193  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [58]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [59]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [60]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.785784770849762nRegular: 13.888874386883515
Ajustado: 13.81181498978195nRegular: 13.90039782504837
Ajustado: 13.54819591748778nRegular: 13.978975725734408
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575363447521962nRegular: 14.079742661574734
Ajustado: 13.562311948859815nRegular: 14.080756222584538
Ajustado: 13.521364643693584nRegular: 14.066559580453122
Ajustado: 13.029458522392627nRegular: 14.216780831796356
Ajustado: 13.70981799580226nRegular: 14.198804602895482
Ajustado: 13.924314685157658nRegular: 14.351734783233747


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [61]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'TVS'],
      dtype='object')

In [62]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [63]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [64]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                  'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                  'Porcentaje_on-base', 'Porcentaje_slugging',
                  'Porcentaje_On-base-plus-slugging']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras', 'WAR', 'Juegos_iniciados',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [65]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
[ 3  4  5  6  7  8  9 10 11 12 13 14]
Pitchers:
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  5 14  2  6  7  8  9 10 11 12 16 17 18]
[ 3  4  

In [66]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [67]:
for year in range(0,period):
    print(year)
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    print("hola")
    # Pitchers:
    for pitcher_name in pitching_indexes:
        print(df_pitching_copy[0].columns[pitcher_name])
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

0
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
1
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
2
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
3
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
4
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
5
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
6
hola
Inning_pitched
Bateos
Carreras
WAR
Juegos_iniciados
Carreras_ganadas
Walks
Strike-outs
Wins
Losses
Saves
WHIP
Dominio
Control
Comando
7
hola
Inning

Apreciemos el resultado final

In [68]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             728 non-null    object 
 1   Juegos                              728 non-null    float64
 2   Porcentaje_juegos                   728 non-null    float64
 3   Juegos_iniciados                    728 non-null    float64
 4   Porcentaje_juegos_iniciados         728 non-null    float64
 5   At-bats                             728 non-null    float64
 6   Bateos                              728 non-null    float64
 7   Dobles                              728 non-null    float64
 8   Triples                             728 non-null    float64
 9   Home-runs                           728 non-null    float64
 10  Runs-batted-in                      728 non-null    float64
 11  Bateos_promedio                     728 non-n

In [69]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             482 non-null    object 
 1   Juegos              482 non-null    float64
 2   Juegos_iniciados    482 non-null    float64
 3   Inning_pitched      482 non-null    float64
 4   Bateos              482 non-null    float64
 5   Carreras            482 non-null    float64
 6   Carreras_ganadas    482 non-null    float64
 7   Walks               482 non-null    float64
 8   Strike-outs         482 non-null    float64
 9   Wins                482 non-null    float64
 10  Losses              482 non-null    float64
 11  Saves               482 non-null    float64
 12  WHIP                482 non-null    float64
 13  ERA                 482 non-null    float64
 14  WAR                 482 non-null    float64
 15  TVS                 482 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [70]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [71]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [72]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [73]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [74]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [75]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [76]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     object 
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [77]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [78]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [79]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 659
Data columns (total 61 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             660 non-null    object 
 1   Juegos                              660 non-null    float64
 2   Porcentaje_juegos                   660 non-null    float64
 3   Juegos_iniciados                    660 non-null    float64
 4   Porcentaje_juegos_iniciados         660 non-null    float64
 5   At-bats                             660 non-null    float64
 6   Bateos                              660 non-null    float64
 7   Dobles                              660 non-null    float64
 8   Triples                             660 non-null    float64
 9   Home-runs                           660 non-null    float64
 10  Runs-batted-in                      660 non-null    float64
 11  Bateos_promedio                     660 non-n

In [80]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 496
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  497 non-null    object 
 1   Juegos                   497 non-null    float64
 2   Juegos_iniciados         497 non-null    float64
 3   Inning_pitched           497 non-null    float64
 4   Bateos                   497 non-null    float64
 5   Carreras                 497 non-null    float64
 6   Carreras_ganadas         497 non-null    float64
 7   Walks                    497 non-null    float64
 8   Strike-outs              497 non-null    float64
 9   Wins                     497 non-null    float64
 10  Losses                   497 non-null    float64
 11  Saves                    497 non-null    float64
 12  WHIP                     497 non-null    float64
 13  ERA                      497 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [81]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [82]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [83]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 56
Data columns (total 66 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            57 non-null     object 
 1   Altura                              57 non-null     float64
 2   Anio                                57 non-null     object 
 3   Anio_de_agente_libre                57 non-null     int64  
 4   Anios_de_contrato_x                 57 non-null     int64  
 5   Anios_de_contrato_y                 57 non-null     int64  
 6   Antiguedad                          57 non-null     int64  
 7   At-bats                             57 non-null     float64
 8   At-bats_2                           57 non-null     float64
 9   Bateos                              57 non-null     float64
 10  Bateos_2                            57 non-null     float64
 11  Bateos_promedio                  

In [84]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 66)
(64, 66)
(81, 66)
(106, 66)
(106, 66)
(133, 66)
(100, 66)
(92, 66)
(103, 66)
(57, 66)
(106, 66)
(55, 66)

 FA - Pitchers:
(0, 73)
(32, 73)
(55, 73)
(84, 73)
(100, 73)
(95, 73)
(69, 73)
(82, 73)
(87, 73)
(82, 73)
(90, 73)
(93, 73)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [85]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [86]:
for year in range(0,period):
    # Condiciones
    condicion_hitter = [df_hitting_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    condicion_pitcher = [df_pitching_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitting_copy[year]['Agente_libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitching_copy[year]['Agente_libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    df_hitting_copy[year] = df_hitting_copy[year].reindex(sorted(df_hitting_copy[year].columns), axis=1)
    df_pitching_copy[year] = df_pitching_copy[year].reindex(sorted(df_pitching_copy[year].columns), axis=1)

In [87]:
for year in range(0,period):
    df_hitting_copy[year].rename(columns = str.lower, inplace = True)
    df_pitching_copy[year].rename(columns = str.lower, inplace = True)

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [88]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Veamos las estadísticas descriptivas de los panel

In [89]:
df_panel_all_hitter[['ln_sueldo_ajustado']].describe()

,ln_sueldo_ajustado
count,7183.000000
mean,13.779294
std,1.647909
min,8.028455
25%,12.693354
50%,13.226723
75%,15.183332
max,17.440000


In [90]:
df_panel_all_pitcher.describe()

,altura,anio_de_agente_libre,anios_de_contrato,antiguedad,bateos,bateos_2,bono_por_firma,cantidad de equipos,cantidad_agentes_libres,carreras,...,whip,whip_2,ws ganadas,walks,walks_2,wins,wins_2,ln_sueldo_ajustado,ln_sueldo_base,ln_sueldo_regular
count,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5.989000e+03,5989.000000,5989.000000,5989.000000,...,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000
mean,6.257741,843.545834,1.469194,0.721322,25.456362,1089.764131,7.673275e+04,2.125397,5.326432,13.523107,...,1.294134,23.957777,3.860077,9.257566,138.939174,1.410685,4.489507,13.503869,13.996677,14.011142
std,0.211227,996.418739,1.202283,0.952433,21.019314,1390.750481,4.980820e+05,1.371459,3.004028,11.474382,...,4.720881,617.612189,5.311391,7.296954,197.101660,1.581105,9.643876,1.524449,1.148616,1.166161
min,5.567014,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.927685,9.835744,9.546813
25%,6.100000,0.000000,1.000000,0.000000,8.180000,66.912400,0.000000e+00,1.000000,3.000000,4.000000,...,0.270000,0.072900,1.000000,3.440000,11.833600,0.230000,0.052900,12.465047,13.178744,13.180632
50%,6.300000,0.000000,1.000000,1.000000,13.680000,187.142400,0.000000e+00,2.000000,5.000000,8.000000,...,0.460000,0.211600,2.000000,6.290000,39.564100,0.770000,0.592900,13.160430,13.262125,13.262125
75%,6.400000,2020.000000,1.000000,1.000000,46.320000,2145.542400,0.000000e+00,2.000000,7.000000,23.820000,...,0.990000,0.980100,5.000000,15.000000,225.000000,2.310000,5.336100,14.626441,14.827111,14.871563
max,6.900000,2029.000000,9.000000,8.000000,120.000000,14400.000000,1.000000e+07,5.000000,18.000000,80.000000,...,210.000000,44100.000000,27.000000,60.000000,3600.000000,20.000000,400.000000,17.584433,17.584433,17.584433


In [91]:
df_panel_all_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7183 entries, 0 to 412
Data columns (total 62 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   acronimo                            7183 non-null   object 
 1   agente_libre                        7183 non-null   object 
 2   altura                              7183 non-null   float64
 3   anio                                7183 non-null   object 
 4   anio_de_agente_libre                7183 non-null   int64  
 5   anios_de_contrato                   7183 non-null   int64  
 6   antiguedad                          7183 non-null   int64  
 7   at-bats                             7183 non-null   float64
 8   at-bats_2                           7183 non-null   float64
 9   bateos                              7183 non-null   float64
 10  bateos_2                            7183 non-null   float64
 11  bateos_promedio                     7183 non

In [92]:
df_panel_all_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5989 entries, 0 to 554
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   acronimo                 5989 non-null   object 
 1   agente_libre             5989 non-null   object 
 2   altura                   5989 non-null   float64
 3   anio                     5989 non-null   object 
 4   anio_de_agente_libre     5989 non-null   int64  
 5   anios_de_contrato        5989 non-null   int64  
 6   antiguedad               5989 non-null   int64  
 7   bateos                   5989 non-null   float64
 8   bateos_2                 5989 non-null   float64
 9   bono_por_firma           5989 non-null   int64  
 10  cantidad de equipos      5989 non-null   int64  
 11  cantidad_agentes_libres  5989 non-null   int64  
 12  carreras                 5989 non-null   float64
 13  carreras_2               5989 non-null   float64
 14  carreras_ganadas         

Verifquemos que no haya problemas con valores *NaN* o *infinitos*

Valores *NaN*:

In [93]:
"""for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"""

"for name in df_panel_all_hitter.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_hitter[name])):\n            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"

In [94]:
"""for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"""

"for name in df_panel_all_pitcher.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_pitcher[name])):\n            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"

Valores *infinitos*

In [95]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if math.isinf(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

acronimo
agente_libre
altura
anio
anio_de_agente_libre
anios_de_contrato
antiguedad
at-bats
at-bats_2
bateos
bateos_2
bateos_promedio
bateos_promedio_2
bono_por_firma
cantidad de equipos
cantidad_agentes_libres
dobles
dobles_2
edad
edad_al_firmar
equipo
estado
ganancias
home-runs
home-runs_2
juegos
juegos totales
juegos_iniciados
juegos_iniciados_2
jugador
pago_efectivo
pennants won
peso
playoffs
porcentaje_on-base-plus-slugging
porcentaje_on-base-plus-slugging_2
porcentaje_juegos
porcentaje_juegos_iniciados
porcentaje_juegos_iniciados_2
porcentaje_on-base
porcentaje_on-base_2
porcentaje_slugging
porcentaje_slugging_2
posicion
promedio_victorias
runs-batted-in
runs-batted-in_2
sueldo_ajustado
sueldo_base
sueldo_porcentual
sueldo_regular
tvs
triples
triples_2
valor_contrato_promedio
valor_contrato_total
valor_del_contrato
victorias
ws ganadas
ln_sueldo_ajustado
ln_sueldo_base
ln_sueldo_regular


In [96]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if math.isinf(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

acronimo
agente_libre
altura
anio
anio_de_agente_libre
anios_de_contrato
antiguedad
bateos
bateos_2
bono_por_firma
cantidad de equipos
cantidad_agentes_libres
carreras
carreras_2
carreras_ganadas
carreras_ganadas_2
comando
comando_2
control
control_2
dominio
dominio_2
era
edad
edad_al_firmar
equipo
estado
ganancias
inning_pitched
inning_pitched_2
juegos
juegos totales
juegos_iniciados
juegos_iniciados_2
jugador
losses
losses_2
pago_efectivo
pennants won
peso
playoffs
porcentaje_juegos
posicion
promedio_victorias
saves
saves_2
strike-outs
strike-outs_2
sueldo_ajustado
sueldo_base
sueldo_porcentual
sueldo_regular
tvs
valor_contrato_promedio
valor_contrato_total
valor_del_contrato
victorias
war
war_2
whip
whip_2
ws ganadas
walks
walks_2
wins
wins_2
ln_sueldo_ajustado
ln_sueldo_base
ln_sueldo_regular


In [97]:
df_panel_all_hitter.sort_index(axis = 1, inplace = True)
df_panel_all_pitcher.sort_index(axis = 1, inplace = True)

Exportemos los paneles

In [98]:
general_visual_path = 'ETL_Data/Panel/Per_10_Games/Visualizations/'

In [99]:
df_panel_all_hitter.to_csv(general_visual_path + 'visual_panel' + '.csv',
                           index = False)
df_panel_all_pitcher.to_csv(general_visual_path + 'visual_panel' + '.csv',
                            index = False)